In [6]:
import numpy as np

class Bee:
    def __init__(self, fitness_func, num_dimensions):
        self.fitness_func = fitness_func
        self.num_dimensions = num_dimensions
        self.position = np.random.uniform(-5, 5, self.num_dimensions)
        self.fitness = self.fitness_func(self.position)
        self.trial_limit = 0
    
    def explore(self):
        new_position = np.clip(self.position + np.random.normal(size=self.num_dimensions), -5, 5)
        new_fitness = self.fitness_func(new_position)
        if new_fitness > self.fitness:
            self.position = new_position
            self.fitness = new_fitness
            self.trial_limit = 0
        else:
            self.trial_limit += 1
    
    def abandon(self):
        self.position = np.random.uniform(-5, 5, self.num_dimensions)
        self.fitness = self.fitness_func(self.position)
        self.trial_limit = 0

class BeeColonyOptimization:
    def __init__(self, fitness_func, num_bees=50, num_iterations=100, elite_sites=2, selected_sites=20, other_sites=28, elite_prob=0.9, other_prob=0.1):
        self.fitness_func = fitness_func
        self.num_dimensions = self.fitness_func
        self.num_bees = num_bees
        self.num_iterations = num_iterations
        self.elite_sites = elite_sites
        self.selected_sites = selected_sites
        self.other_sites = other_sites
        self.elite_prob = elite_prob
        self.other_prob = other_prob
    
    def run(self):
        # Initialize the population of bees
        bees = [Bee(self.fitness_func, self.num_dimensions) for i in range(self.num_bees)]
        
        # Initialize the best solution and fitness
        best_solution = np.copy(bees[0].position)
        best_fitness = bees[0].fitness
        
        # Run the Bee Colony Optimization algorithm
        for iteration in range(self.num_iterations):
            # Employed bees phase
            for i in range(self.num_bees):
                bee = bees[i]
                bee.explore()
                if bee.fitness > best_fitness:
                    best_solution = np.copy(bee.position)
                    best_fitness = bee.fitness
            
            # Onlooker bees phase
            elite_bees = sorted(bees, key=lambda bee: bee.fitness, reverse=True)[:self.elite_sites]
            selected_bees = np.random.choice(bees, size=self.selected_sites, replace=False)
            other_bees = [bee for bee in bees if bee not in elite_bees and bee not in selected_bees]
            
            total_fitness = sum([bee.fitness for bee in selected_bees])
            probabilities = [bee.fitness / total_fitness for bee in selected_bees]
            
            for i in range(self.selected_sites):
                bee = selected_bees[i]
                if np.random.rand() < self.elite_prob:
                    elite_bee = np.random.choice(elite_bees)
                    new_position = bee.position + self.elite_prob * (bee.position - elite_bee.position)
                else:
                    other_bee = np.random.choice(other_bees)
                    new_position = bee.position + self.other_prob * (bee.position - other_bee.position)
                new_fitness = self.fitness_func(new_position)
                if new_fitness > bee.fitness:
                    bee.position = new_position
                    bee.fitness = new_fitness
                    bee.trial_limit = 0
                else:
                    bee.trial_limit += 1


In [7]:
import numpy as np

def fitness_function(x):
    return np.sum(x**4)

# Define the bounds of the search space
bounds = np.array([[-5, 5], [-5, 5]])

# Create a random solution within the search space
x = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(1, 2))

# Evaluate the fitness function on the random solution
fitness = fitness_function(x)

# Calculate the dimension of the fitness value
fitness_dim = np.array([fitness]).ndim

print(f"Fitness value: {fitness}")
print(f"Fitness dimension: {fitness_dim}")


aco = BeeColonyOptimization(fitness_function, num_bees=100, num_iterations=1000,elite_sites=2, selected_sites=20, other_sites=28, elite_prob=0.9, other_prob=0.1)
best_solution, best_fitness = aco.run()
print(f"Best solution: {best_solution}")
print(f"Best fitness: {best_fitness}")

Fitness value: 34.13760661521512
Fitness dimension: 1


TypeError: expected sequence object with len >= 0 or a single integer